In [1]:
import getpass
import pandas as pd
import psycopg2 
import json
import requests
from datetime import datetime
import numpy as np

# Variables
weekly_payout = 15.51

# Define PostgreSQL database connection parameters
# user = input("username")

api_key = getpass.getpass("Enter DataGolf API key:")
host = getpass.getpass("Enter Database Host:")
port = "5432" # The default port for PosgreSQL Server
dbname = 'postgres'
user = getpass.getpass("Enter Username:")
password = getpass.getpass("Enter Password:")

# Define a SQLAlchemy URI string for connecting to the database
# The URI structure is [DB_FLAVOR]+[DB_PYTHON_LIBRARY]://[USERNAME]:[PASSWORD]@[DB_HOST]:[PORT]/[DB_NAME]
db_URI = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"

In [ ]:
# A helper function to open a psycopg2 connection, set auto-commit to true, execute the sql, and close the connection.
# This will mainly be used for writing to the db
# def execute_sql(sql, echo=False):
#     try:
#         pg_conn = psycopg2.connect(
#             dbname=dbname,
#             user=user,
#             password=password,
#             host=host,
#             port=port
#         )
#     except psycopg2.Error as e:
#         error_message = e.pgerror
#         print("Error Connecting:", error_message)
   
#     try:
#         # Set the connection to autocommit (everything is treated as an individual transaction)
#         pg_conn.set_session(autocommit=True)
        
#         # The cursor is used to execute ddl statements.
#         pg_cursor = pg_conn.cursor() 

#         pg_cursor.execute(sql)
#         if echo:
#             print(sql)

#         results = pg_cursor.fetchall()
#         return results
    
#     except psycopg2.Error as e:
#         error_message = e.pgerror
#         print("SQL Failed:", error_message)
#         return []
    
#     finally:
#         if pg_cursor:
#             pg_cursor.close()
#         if pg_conn:
#             pg_conn.close()

In [2]:
#function for uploading df to aws postgre SQL database
def insert_df_to_sql(table_name, df):
    try:
        pg_conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
    except psycopg2.Error as e:
        error_message = e.pgerror
        print("Error Connecting:", error_message)

    try:
        # Create a cursor object
        cursor = pg_conn.cursor()

        # Convert the DataFrame to a list of tuples for insertion
        insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"
        data_to_insert = df.to_records(index=False).tolist()

        # Insert data using executemany()
        cursor.executemany(insert_query, data_to_insert)

        # Commit the transaction
        pg_conn.commit()
        print("Data inserted successfully!")
    except psycopg2.Error as e:
        error_message = e.pgerror
        print("Exception uploading to "+table_name+" table."+error_message)

    finally:
        # Close the cursor and connection
        cursor.close()
        pg_conn.close()


In [ ]:
# def psyco_read_sql(sql):
#     try:
#         pg_conn = psycopg2.connect(
#             dbname=dbname,
#             user=user,
#             password=password,
#             host=host,
#             port=port
#         )
#     except psycopg2.Error as e:
#         error_message = e.pgerror
#         print("Error Connecting:", error_message)

#     # Use pd.read_sql to execute the query and load data into a DataFrame
#     df = pd.read_sql(sql, pg_conn)

#     # Print the DataFrame
#     print(df)

#     # Close the database connection
#     pg_conn.close()


In [3]:
#function to gather df from json response
def df_from_json(feed):
    response = requests.get(feed)
    json_data = response.json()
    df = pd.DataFrame(json_data)
    return df

# Check for missing players

In [4]:
#get the players list from datagolf
players_dg = df_from_json('https://feeds.datagolf.com/get-player-list?file_format=json&key='+api_key)

#get the unique dg_id from players already in aws db
players_aws = pd.read_sql(sql = """
                          select distinct dg_id from player;
                          """, con=db_URI)

In [5]:
#find players that are in the datagolf db that are missing from the aws db
missing_players = players_dg[~players_dg['dg_id'].isin(players_aws['dg_id'])] # ~ operator inverts - this is looking for all golfers in datagolf db that are not in aws
missing_players

,amateur,country,country_code,dg_id,player_name
284,1,Austria,AUT,26456,"Boandl, Lukas"
487,0,United States,USA,33366,"Cassidy, Nic"
716,0,United States,USA,32461,"Davidson, Ethan"
760,1,United States,USA,33841,"Dean, Kellen"
773,0,Austria,AUT,21657,"Denk, Luca"
833,1,Netherlands,NED,31545,"Driessen, Bjorn"
911,1,Austria,AUT,26457,"Ennsmann, David"
915,0,Spain,ESP,33839,"Ereno Perez, Pablo"
926,1,Belgium,BEL,33651,"Estas, Arthur"
945,0,Italy,ITA,27784,"Fallotico, Lucas Nicolas"


In [6]:
#if there's missing players, add them to the aws db
if(len(missing_players['player_name'])>0):
    insert_df_to_sql('player', missing_players)

Data inserted successfully!


# Check for Event Updates

In [7]:
#dynamic current year so script can be re-used next year
current_year = datetime.now().year

#capture datagolf events and the current year events already uploaded to aws
events_dg = df_from_json('https://feeds.datagolf.com/historical-raw-data/event-list?file_format=json&key='+api_key)
events_aws_current_year = pd.read_sql(sql = f"""
                          select event_id from event where calendar_year = {current_year};
                          """, con=db_URI)

#filter datagolf historical events to current year, pga
events_dg_current_year = events_dg[(events_dg['tour'] == 'pga') & (events_dg['calendar_year'] == events_dg['calendar_year'].max())]

In [8]:
#find which events are not in aws
missing_events = events_dg_current_year[~events_dg_current_year['event_id'].isin(events_aws_current_year['event_id'])]
missing_events

,calendar_year,date,event_id,event_name,sg_categories,tour,traditional_stats
1,2025,2025-07-06,30,John Deere Classic,yes,pga,yes
12,2025,2025-06-29,524,Rocket Classic,yes,pga,yes
31,2025,2025-06-22,34,Travelers Championship,yes,pga,yes
39,2025,2025-06-15,26,U.S. Open,yes,pga,yes
52,2025,2025-06-08,32,RBC Canadian Open,yes,pga,yes
60,2025,2025-06-01,23,the Memorial Tournament presented by Workday,yes,pga,yes
80,2025,2025-05-25,21,Charles Schwab Challenge,yes,pga,yes
123,2025,2025-04-27,18,Zurich Classic of New Orleans,yes,pga,basic


# Check for DFS event

In [9]:
#create array to hold missing dfs event ids
no_dfs = []

#loop through each missing event to check for empty response code 400
for id_ev in missing_events['event_id']:
    feed = 'https://feeds.datagolf.com/historical-dfs-data/points?tour=pga&site=fanduel&event_id='+str(id_ev)+'&year='+str(current_year)+'&file_format=json&key='+api_key
    response = requests.get(feed)
    if(response.status_code == 400):
        no_dfs.append(id_ev)

#remove any missing events from missing_events df as the DFS event may not be ready yet (or will never exist)
missing_events = missing_events[~missing_events['event_id'].isin(no_dfs)]
missing_events

,calendar_year,date,event_id,event_name,sg_categories,tour,traditional_stats
1,2025,2025-07-06,30,John Deere Classic,yes,pga,yes
12,2025,2025-06-29,524,Rocket Classic,yes,pga,yes
31,2025,2025-06-22,34,Travelers Championship,yes,pga,yes
39,2025,2025-06-15,26,U.S. Open,yes,pga,yes
52,2025,2025-06-08,32,RBC Canadian Open,yes,pga,yes
60,2025,2025-06-01,23,the Memorial Tournament presented by Workday,yes,pga,yes
80,2025,2025-05-25,21,Charles Schwab Challenge,yes,pga,yes


In [10]:
#if there's any missing events, add them to the aws db
if(len(missing_events['event_id'])>0):
    missing_events_w_dfs_payout = missing_events.copy().drop(['sg_categories', 'tour', 'traditional_stats'], axis=1)
    missing_events_w_dfs_payout['dfs_payout'] = None
    insert_df_to_sql('event', missing_events_w_dfs_payout)

Data inserted successfully!


In [11]:
pd.read_sql("""select * from event where calendar_year = 2025 order by date desc""", con=db_URI)

,id_event,calendar_year,event_id,date,event_name,dfs_payout
0,149f66dc-3ab9-41cd-beac-8ae9f4cce4ec,2025,30,2025-07-06,John Deere Classic,NaN
1,2428ab4c-f237-4710-87c8-a590a1801c86,2025,524,2025-06-29,Rocket Classic,NaN
2,d31270be-c8f7-4943-9617-a02c164009a6,2025,34,2025-06-22,Travelers Championship,NaN
3,b5740648-8d2c-4e8c-b77c-7a4ede7e131d,2025,26,2025-06-15,U.S. Open,NaN
4,8629a0d0-3267-485f-9407-2ae1a6f36b95,2025,32,2025-06-08,RBC Canadian Open,NaN
5,2f4ae57c-5761-4389-938f-faccade83baf,2025,23,2025-06-01,the Memorial Tournament presented by Workday,NaN
6,917ab760-d513-4bc2-bc2c-278247e3d0b8,2025,21,2025-05-25,Charles Schwab Challenge,NaN
7,01e63ba9-5229-4d9e-8173-7e18522e31f1,2025,33,2025-05-18,PGA Championship,15.51
8,9ee2c520-9f83-42ff-87eb-0bbd45558ef8,2025,553,2025-05-11,ONEflight Myrtle Beach Classic,NaN
9,0f4e6a0c-603f-4e84-9305-3627990aa23e,2025,480,2025-05-11,Truist Championship,15.51


# Parse Raw Player Scoring & Data

In [12]:
#get headers from an event with everthing documented (event 2 in 2025 is one example)
event_for_names = df_from_json('https://feeds.datagolf.com/historical-raw-data/rounds?tour=pga&event_id=2&year=2025&file_format=json&key='+api_key)
basic_names = ['event_id', 'calendar_year', 'dg_id', 'round', 'fin_text']
stat_names = basic_names + pd.json_normalize(event_for_names['scores'].loc[0]['round_1'], max_level=0).columns.tolist()
stat_names.sort()

#create placeholder array to store round information.  *Getting warnings appending to pre-set dataframe so went the array route for now.  Could potentially pre-define/pre-fill df with None but this gets tricky because some rounds will not be uploaded due to CUT or WD
rounds = []

for id_ev in missing_events['event_id']:
    #get the event historical data
    event_temp = df_from_json('https://feeds.datagolf.com/historical-raw-data/rounds?tour=pga&event_id='+str(id_ev)+'&year='+str(current_year)+'&file_format=json&key='+api_key)
    
    #loop through players
    player_summary = pd.json_normalize(event_temp['scores'], max_level=0)
    for row in player_summary.iterrows():
        player = row[1]

        #create array of round info.  If there is a missing round, do not add
        event_rounds = []
        if pd.notna(player['round_1']):
            event_rounds = event_rounds + [pd.json_normalize(player['round_1'], max_level = 0)]
        if pd.notna(player['round_2']):
            event_rounds = event_rounds + [pd.json_normalize(player['round_2'], max_level = 0)]
        if pd.notna(player['round_3']):
            event_rounds = event_rounds + [pd.json_normalize(player['round_3'], max_level = 0)]
        if pd.notna(player['round_4']):
            event_rounds = event_rounds + [pd.json_normalize(player['round_4'], max_level = 0)]

        for index, rnd in enumerate(event_rounds):
            #add player summary info (basic_names) to each round df and add to rounds
            basic_cols = pd.DataFrame({'dg_id':[player['dg_id']], 'fin_text':[player['fin_text']], 'round':[index+1], 'event_id':[id_ev], 'calendar_year':[current_year]})
            rnd[basic_cols.columns] = basic_cols

            #find differences in the column headers (if any - some events do not contain all the data, so will need to fill with NA)
            diffs = set(stat_names)-set(rnd.columns)

            #add NA to missing columns
            for item in diffs:
                rnd[item] = None

            #sort the columns so array can be easily converted to df later
            rnd_sorted = rnd.copy()[sorted(rnd.columns)]

            #append array
            rounds.append(rnd_sorted.loc[0].values)

#convert to dataframe and normalize NAs
rounds = pd.DataFrame(rounds, columns=stat_names).replace([np.nan, 'missing'], None, inplace=False)
rounds.sample(10)

,birdies,bogies,calendar_year,course_name,course_num,course_par,dg_id,doubles_or_worse,driving_acc,driving_dist,...,score,scrambling,sg_app,sg_arg,sg_ott,sg_putt,sg_t2g,sg_total,start_hole,teetime
2003,4,4,2025,TPC Toronto at Osprey Valley (North Course),935,70,30892,0,0.643,296.6,...,70,0.5,-0.07,-0.236,-0.237,0.073,-0.543,-0.471,1,8:24am
1828,3,1,2025,TPC Toronto at Osprey Valley (North Course),935,70,22760,0,0.786,295.6,...,68,0.75,1.302,-0.256,0.903,-1.195,1.949,0.755,1,1:17pm
2156,5,3,2025,Muirfield Village Golf Club,23,72,12294,1,0.857,289.2,...,72,0.667,-0.4,0.312,1.677,0.355,1.589,1.944,1,12:35pm
2346,8,1,2025,Colonial Country Club,21,70,24968,0,0.643,313.2,...,63,1.0,3.951,0.819,0.842,1.202,5.612,6.813,10,8:17am
1473,4,4,2025,Oakmont Country Club,608,70,15856,1,0.5,295.9,...,73,0.667,-1.002,1.81,-0.234,1.224,0.573,1.781,1,1:47pm
2577,3,1,2025,Colonial Country Club,21,70,27819,0,0.643,292.4,...,68,0.75,0.285,0.568,0.856,0.634,1.709,2.343,10,8:50am
826,3,2,2025,Detroit Golf Club,876,72,17538,0,0.643,290.9,...,71,0.5,0.566,-0.603,-0.932,-0.928,-0.969,-1.897,1,12:54pm
2042,3,4,2025,TPC Toronto at Osprey Valley (North Course),935,70,23502,0,0.571,287.4,...,71,0.333,-0.258,0.125,-0.789,-1.324,-0.922,-2.245,1,8:24am
2245,1,0,2025,Muirfield Village Golf Club,23,72,15556,0,0.714,280.1,...,71,0.857,1.183,0.922,0.152,-0.298,2.257,1.958,1,10:40am
1889,6,2,2025,TPC Toronto at Osprey Valley (North Course),935,70,14926,0,0.643,294.5,...,66,0.75,0.578,-0.812,-0.48,4.244,-0.714,3.529,10,7:51am


## Upload Any Missing Courses

In [13]:
new_courses = rounds.drop_duplicates('course_num')[['course_name', 'course_num', 'course_par']].reset_index().drop('index', axis=1)
courses_aws = pd.read_sql(sql="""SELECT * FROM COURSE;""", con=db_URI)
missing_courses = new_courses[~new_courses['course_num'].isin(courses_aws['course_num'])]
missing_courses

,course_name,course_num,course_par
3,Oakmont Country Club,608,70
4,TPC Toronto at Osprey Valley (North Course),935,70


In [14]:
#if there's missing courses, add them to the aws db
if(len(missing_courses['course_num'])>0):
    insert_df_to_sql('course', missing_courses)

Data inserted successfully!


## Upload Player Scoring

In [15]:
#cleanup rounds df
rounds_clean = rounds.drop(['course_name', 'course_par', 'fin_text'], axis=1)

#get necessary event information
events_aws = pd.read_sql(sql="""SELECT id_event, event_id, calendar_year FROM EVENT WHERE CALENDAR_YEAR = 2025;""", con=db_URI)
events_aws['id_event'] = events_aws['id_event'].astype(str)
events_aws

#get necessary course information
courses_aws = pd.read_sql(sql="""SELECT id_course, course_num FROM COURSE;""", con=db_URI)

#replace aws id's with dg
rounds_clean = pd.merge(rounds_clean, events_aws, on=['event_id', 'calendar_year'], how='left').drop(['event_id', 'calendar_year'], axis=1)
rounds_clean = pd.merge(rounds_clean, courses_aws, on=['course_num'], how='left').drop('course_num', axis=1)

#preview
rounds_clean.sample(10)

,birdies,bogies,dg_id,doubles_or_worse,driving_acc,driving_dist,eagles_or_better,gir,great_shots,pars,...,sg_app,sg_arg,sg_ott,sg_putt,sg_t2g,sg_total,start_hole,teetime,id_event,id_course
1064,4,2,23465,0,0.571,298.6,1,0.778,4,11,...,0.288,0.548,1.519,-0.354,2.355,2.0,1,12:10pm,d31270be-c8f7-4943-9617-a02c164009a6,67e79442-5183-44cd-b1bb-402ec103f38f
294,6,5,32330,0,0.5,309.3,0,0.833,3,7,...,1.932,-0.069,0.792,-3.079,2.655,-0.423,10,8:35am,149f66dc-3ab9-41cd-beac-8ae9f4cce4ec,06ba465b-fb0b-4c30-8ef8-d043f50204cf
242,6,3,21756,0,0.429,302.8,0,0.667,4,9,...,0.808,-0.944,-0.728,2.756,-0.864,1.892,10,9:00am,149f66dc-3ab9-41cd-beac-8ae9f4cce4ec,06ba465b-fb0b-4c30-8ef8-d043f50204cf
2147,4,4,19483,0,0.714,290.5,0,0.611,4,10,...,-0.526,-1.085,0.248,2.1,-1.363,0.737,1,12:59pm,2f4ae57c-5761-4389-938f-faccade83baf,d6ddae91-c30d-4998-a551-88dba1012a6a
1143,2,5,29767,0,0.429,283.7,1,0.5,4,10,...,0.941,0.746,-1.065,-0.936,0.622,-0.314,1,8:30am,d31270be-c8f7-4943-9617-a02c164009a6,67e79442-5183-44cd-b1bb-402ec103f38f
2252,1,3,10419,0,0.714,285.9,0,0.722,2,14,...,-0.826,0.703,0.918,-0.851,0.795,-0.056,1,8:50am,2f4ae57c-5761-4389-938f-faccade83baf,d6ddae91-c30d-4998-a551-88dba1012a6a
1805,2,1,23504,0,0.786,306.2,0,0.667,1,15,...,-0.76,-0.071,1.642,-0.281,0.811,0.529,10,12:11pm,8629a0d0-3267-485f-9407-2ae1a6f36b95,41b84541-e469-4407-8d27-0618f9181ead
1203,4,6,17536,0,0.5,280.6,0,0.667,5,8,...,-0.528,0.16,-1.086,2.665,-1.453,1.212,1,2:04pm,b5740648-8d2c-4e8c-b77c-7a4ede7e131d,9b220dd9-83ac-4f67-86ec-e1a96e434836
2706,1,3,12423,1,0.643,293.1,0,0.5,4,13,...,-0.465,-2.074,-0.211,-1.436,-2.75,-4.187,1,12:45pm,917ab760-d513-4bc2-bc2c-278247e3d0b8,eedd6e23-a4b5-43ce-9745-7b5fcad7aeda
532,3,3,14140,0,0.714,302.9,0,0.667,2,12,...,-0.942,-0.625,0.2,-0.436,-1.367,-1.802,1,11:15am,2428ab4c-f237-4710-87c8-a590a1801c86,689582fd-0d30-41a6-89a8-344350374c90


In [16]:
insert_df_to_sql('round', rounds_clean)

Data inserted successfully!


# Upload Player DFS

In [17]:
#get all stat headers for dfs scoring (they all exist for event 2 in 2025)
dfs = df_from_json('https://feeds.datagolf.com/historical-dfs-data/points?tour=pga&site=fanduel&event_id=2&year=2025&file_format=json&key='+api_key)
stat_names = pd.json_normalize(dfs['dfs_points'].loc[0], max_level=0).drop(['ownership', 'player_name'], axis=1).columns.tolist()+['id_event']
stat_names.sort()

#set placeholder array
dfs_array = []

#loop through the missing events
for id_ev in missing_events['event_id']:

    #get the event historical data
    event_temp = df_from_json('https://feeds.datagolf.com/historical-dfs-data/points?tour=pga&site=fanduel&event_id='+str(id_ev)+'&year='+str(current_year)+'&file_format=json&key='+api_key)

    #loop through golfer's dfs event summary
    for player in event_temp['dfs_points']:
        #get the dfs points, add calendar year and event id so it can be merged with the aws id_event
        dfs_pts = pd.json_normalize(player, max_level=0).drop(['player_name', 'ownership'], axis=1)
        dfs_pts['calendar_year'] = current_year
        dfs_pts['event_id'] = id_ev
        dfs_pts = pd.merge(dfs_pts, events_aws, on=['event_id', 'calendar_year']).drop(['calendar_year', 'event_id'], axis=1)
        
        #find differences in the column headers (if any - some events do not contain all the data, so will need to fill with NA)
        diffs = set(stat_names)-set(dfs_pts.columns)

        #add NA to missing columns
        for item in diffs:
            dfs_pts[item] = None
    
        #sort the columns so array can be easily converted to df later
        dfs_pts_sorted = dfs_pts.copy()[sorted(dfs_pts.columns)]

        #append to placeholder array
        dfs_array.append(dfs_pts_sorted.loc[0].values)

#convert to dataframe and normalize NAs
dfs_array = pd.DataFrame(dfs_array, columns=stat_names).replace([np.nan, 'missing'], None, inplace=False)
dfs_array.sample(10)
    

,bogey_free_pts,bounce_back_pts,dg_id,fin_text,finish_pts,five_birdie_pts,hole_score_pts,id_event,salary,streak_pts,total_pts
592,0,1.2,16341,T52,0,4,66.1,8629a0d0-3267-485f-9407-2ae1a6f36b95,7000,1.8,73.1
738,0,1.2,14140,T51,0,4,37.1,2f4ae57c-5761-4389-938f-faccade83baf,7000,0.6,42.9
169,5,0.6,14704,T8,8,8,90.0,2428ab4c-f237-4710-87c8-a590a1801c86,8300,4.8,116.4
640,0,0.0,23890,CUT,0,0,28.7,8629a0d0-3267-485f-9407-2ae1a6f36b95,7000,1.2,29.9
122,0,0.3,17273,CUT,0,0,25.0,149f66dc-3ab9-41cd-beac-8ae9f4cce4ec,7000,0.6,25.9
443,0,0.3,19870,T57,0,0,17.6,b5740648-8d2c-4e8c-b77c-7a4ede7e131d,8500,0.6,18.5
179,5,0.0,14140,T24,3,12,84.2,2428ab4c-f237-4710-87c8-a590a1801c86,7100,2.4,106.6
878,0,0.0,28469,CUT,0,0,16.5,917ab760-d513-4bc2-bc2c-278247e3d0b8,8200,0.0,16.5
492,0,0.3,18079,CUT,0,0,4.2,b5740648-8d2c-4e8c-b77c-7a4ede7e131d,10900,0.0,4.5
897,0,0.0,159,CUT,0,0,1.6,917ab760-d513-4bc2-bc2c-278247e3d0b8,7000,0.0,1.6


In [18]:
insert_df_to_sql('dfs_total', dfs_array)

Data inserted successfully!
